# Método 2: Probabilistic Matrix Factorization

In [10]:
import random
from funciones import preprocesar_dataframe_animes, \
                      get_metricas

In [11]:
(train_df, test_df, 
 ratings_train_matrix, ratings_test_matrix,
 NUM_USERS, NUM_ANIMES, MIN_RATING, MAX_RATING, SCORES) = \
    preprocesar_dataframe_animes(
    dataframe_path='csv/rating.csv',
    n_user_ratings=5,
    n_anime_ratings=5,
    num_users=1000,
    test_size=0.2, 
    RANDOM_STATE=42 
    )

In [12]:
train_df

,user_id,anime_id,rating
90138,840,421,10
78068,739,1822,6
59641,568,548,10
41391,383,948,4
15836,150,451,10
...,...,...,...
48297,437,2242,10
2321,30,818,8
62304,597,304,10
68211,647,632,7


Definimos los parámetros para la factorización matricial mediante PMF.

Hemos probado con distintos num_factores y no hay mucha diferencia entre 5, 7, 10, 16 y 32. Lo mismo con Regularization y Learning Rate, probamos distintos parámetros y el cambio era mínimo.

In [14]:
NUM_FACTORS = 2       # Número de factores latentes.
LEARNING_RATE = 0.001 # Tasa de aprendizaje (gamma).
REGULARIZATION = 0.01  # Parámetro de regularización(lambda).
NUM_ITERATIONS = 100   # Número de iteraciones en el proceso de entrenamiento.

In [15]:
def compute_biased_prediction (avg, b_u, b_i, p_u, q_i):
  deviation = 0
  for k in range(NUM_FACTORS):
    deviation += p_u[k] * q_i[k]

  prediction = avg + b_u + b_i + deviation
  return prediction

Ahora vamos a inicializar las matrices de factores.

- `p` es la matriz de factores de los usuarios.
- `q` es la matriz de factores de los animes (los items).

Y también los vectores de bias.

- `bu` es el bias del usuario u.
- `ba` es el bias del anime a.

In [16]:
p = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)]
q = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ANIMES)]

bu = [random.random() for _ in range(NUM_USERS)]
bi = [random.random() for _ in range(NUM_ANIMES)]

Calculamos el voto medio.

In [17]:
rating_average = 0
rating_count = 0

for u in range(NUM_USERS):
  for i in range(NUM_ANIMES):
    if ratings_train_matrix[u][i] != None:
      rating_average += ratings_train_matrix[u][i]
      rating_count += 1

rating_average /= rating_count

Ahora entrenamos nuestro modelo.

In [18]:
for it in range(NUM_ITERATIONS):
  print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))

  updated_p = list(p) # clone p matrix
  updated_q = list(q) # clone q matrix

  updated_bu = list(bu) # clone bu vector
  updated_bi = list(bi) # clone bi vector

  for u in range(NUM_USERS):
    for i in range(NUM_ANIMES):
      if ratings_train_matrix[u][i] != None:

        prediction = compute_biased_prediction(rating_average, bu[u], bi[i], p[u], q[i])
        rating = ratings_train_matrix[u][i]
        error = rating - prediction

        for k in range(NUM_FACTORS):
          updated_p[u][k] += LEARNING_RATE * (error * q[i][k] - REGULARIZATION * p[u][k])
          updated_q[i][k] += LEARNING_RATE * (error * p[u][k] - REGULARIZATION * q[i][k])

        updated_bu[u] += LEARNING_RATE * (error - REGULARIZATION * bu[u])
        updated_bi[i] += LEARNING_RATE * (error - REGULARIZATION * bi[i])


  p = updated_p
  q = updated_q

  bu = updated_bu
  bi = updated_bi

Iteración 1 de 100
Iteración 2 de 100
Iteración 3 de 100
Iteración 4 de 100
Iteración 5 de 100
Iteración 6 de 100
Iteración 7 de 100
Iteración 8 de 100
Iteración 9 de 100
Iteración 10 de 100
Iteración 11 de 100
Iteración 12 de 100
Iteración 13 de 100
Iteración 14 de 100
Iteración 15 de 100
Iteración 16 de 100
Iteración 17 de 100
Iteración 18 de 100
Iteración 19 de 100
Iteración 20 de 100
Iteración 21 de 100
Iteración 22 de 100
Iteración 23 de 100
Iteración 24 de 100
Iteración 25 de 100
Iteración 26 de 100
Iteración 27 de 100
Iteración 28 de 100
Iteración 29 de 100
Iteración 30 de 100
Iteración 31 de 100
Iteración 32 de 100
Iteración 33 de 100
Iteración 34 de 100
Iteración 35 de 100
Iteración 36 de 100
Iteración 37 de 100
Iteración 38 de 100
Iteración 39 de 100
Iteración 40 de 100
Iteración 41 de 100
Iteración 42 de 100
Iteración 43 de 100
Iteración 44 de 100
Iteración 45 de 100
Iteración 46 de 100
Iteración 47 de 100
Iteración 48 de 100
Iteración 49 de 100
Iteración 50 de 100
Iteración

Calculamos las predicciones para todos los items que haya recibido una votación de test:

In [19]:
predictions = [[None for _ in range(NUM_ANIMES)] for _ in range(NUM_USERS)]

for u in range(NUM_USERS):
  for i in range(NUM_ANIMES):
    if ratings_test_matrix[u][i] != None:
      predictions[u][i] = compute_biased_prediction(rating_average, bu[u], bi[i], p[u], q[i])

Obtenemos las métricas.

In [ ]:
# 100 K RATINGS ENTRENAMIENTO
# 40 ANIMES MIN Y 40 USUARIOS MIN
# 10 RECOMENDACIONES THETA = 7

get_metricas(ratings_test_matrix, NUM_ANIMES, NUM_USERS,
             predictions, 7, 10)

MAE =  0.9246024329416812
RMSE =  1.1537824555524556
Precision =  0.9221297619047587
Recall =  0.5682123876346582
F1 =  0.6521459897901318


In [ ]:
# 100 K RATINGS ENTRENAMIENTO
# 40 ANIMES MIN Y 40 USUARIOS MIN
# 5 RECOMENDACIONES THETA = 7

get_metricas(ratings_test_matrix, NUM_ANIMES, NUM_USERS,
             predictions, 7, 5)

MAE =  0.9246024329416812
RMSE =  1.1537824555524556
Precision =  0.9466666666666643
Recall =  0.3096209621479688
F1 =  0.43566036224052956


# Nuestro modelo para las métricas

In [26]:
# 100K RATINGS ENTRENAMIENTO
# 5 ANIMES MIN Y 5 USUARIOS MIN
get_metricas(ratings_test_matrix, NUM_ANIMES, NUM_USERS,
             predictions, 7, 5)

MAE =  0.9314183742273114
RMSE =  1.127376965191136
Precision =  0.9354666666666654
Recall =  0.5457409308811286
F1 =  0.6151643122072391


In [25]:
from funciones import get_ndcg
get_ndcg(ratings_test_matrix, NUM_ANIMES, NUM_USERS, predictions, 5)

0.6318250483174597

Vemos que este método logra una gran precisión en general.

# Extra:

Aunque nuestras pruebas son con tan solo 5 recomendaciones, queríamos demostrar que en este modelo en específico al recomendar 10 animes lo hace bastante bien y con un ndcg bastante alto.

In [71]:
# 100K RATINGS ENTRENAMIENTO
# 5 ANIMES MIN Y 5 USUARIOS MIN
get_metricas(ratings_test_matrix, NUM_ANIMES, NUM_USERS,
             predictions, 7, 10)

MAE =  0.983056802860995
RMSE =  1.2002274211754953
Precision =  0.9141645613867814
Recall =  0.713685245626937
F1 =  0.7507595686567121


In [24]:
from funciones import get_ndcg
get_ndcg(ratings_test_matrix, NUM_ANIMES, NUM_USERS, predictions, 10)

0.7460843459579647